In [1]:
import pandas as pd
import seaborn as sns
import statistics as stat
import numpy as np
import matplotlib.pyplot as plt
import glob
from os import listdir
from os.path import isfile, join
from PIL import Image
import bpy
import random
import cv2
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
from skimage import color
import keras

In [2]:
X_train_file = np.load('X_train_file.npy', allow_pickle=True)
X_test_file = np.load('X_test_file.npy', allow_pickle=True)
X_valid_file = np.load('X_valid_file.npy', allow_pickle=True)


X_train = X_train_file.item()['data']
y_train = X_train_file.item()['label'] 
X_test = X_test_file.item()['data']
y_test = X_test_file.item()['label'] 
X_valid = X_valid_file.item()['data']
y_valid = X_valid_file.item()['label'] 

In [3]:
print('X_train shape:'+str(X_train.shape), 'y_train shape:'+str(y_train.shape))
print('X_test shape:'+str(X_test.shape), 'y_test shape:'+str(y_test.shape))
print('X_valid shape:'+str(X_valid.shape), 'y_valid shape:'+str(y_valid.shape))

X_train shape:(45980, 56, 56, 3) y_train shape:(45980, 315)
X_test shape:(1575, 56, 56, 3) y_test shape:(1575, 315)
X_valid shape:(1575, 56, 56, 3) y_valid shape:(1575, 315)


In [5]:
# Curious I wanted to see how my model would do with higher pixel count

I just wanted to check how our previous model does with larger image size 

In [23]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(56, 56, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(315, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics="accuracy")



In [24]:
callback = [EarlyStopping(patience=3, monitor='val_accuracy', restore_best_weights=True),
            ReduceLROnPlateau(monitor = 'val_loss', patience = 1, factor=0.5, verbose=1)]

In [25]:

model_hist = model.fit(X_train, y_train,  callbacks=callback, epochs=100, validation_data=(X_test, y_test))

_, acc = model.evaluate(images['test'], y_test, verbose=1)
print('Test Accuracy: %.3f' % (acc * 100))

Epoch 1/100
1437/1437 [==============================] - 20s 9ms/step - loss: 5.2084 - accuracy: 0.0321 - val_loss: 4.2501 - val_accuracy: 0.1054 - lr: 0.0010
Epoch 2/100
1437/1437 [==============================] - 12s 8ms/step - loss: 4.0234 - accuracy: 0.1539 - val_loss: 3.1015 - val_accuracy: 0.2724 - lr: 0.0010
Epoch 3/100
 848/1437 [================>.............] - ETA: 4s - loss: 3.2895 - accuracy: 0.2702

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(model_hist.history['accuracy'])
ax[0].plot(model_hist.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')


ax[1].plot(model_hist.history['loss'])
ax[1].plot(model_hist.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper left')
plt.show()

In [26]:
from numba import cuda

cuda.select_device(0)
cuda.close()